In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install openai
!pip install flash-attn --no-build-isolation
!apt-get install fonts-noto-cjk  # For Noto Sans CJK JP


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vrsap5kx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vrsap5kx
  Resolved https://github.com/huggingface/transformers to commit d1b92369ca193da49f9f7ecd01b08ece45c2c9aa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11203014 sha256=117163195a1557cc29044230d176bfa5647df818b0c85cd0da254dbf3f50eead
  Stored in directory: /tmp/pip-ephem-wheel-cache-b9bpoyf7/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.0
    Uninstalling transformers-4.51.0:
      Successfully uninstalled transformers-4.51.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:


import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
model_path = "Qwen/Qwen2.5-VL-3B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2",device_map="auto")
processor = AutoProcessor.from_pretrained(model_path)


2025-04-06 08:26:43.191611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743928003.209697   19608 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743928003.215201   19608 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743928003.229653   19608 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743928003.229675   19608 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743928003.229677   19608 computation_placer.cc:177] computation placer alr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [15]:
# df= None

In [11]:

import os
import io
import pandas as pd
from PIL import Image as Image_PIL
import shutil
from datasets import load_dataset, Dataset, Image
from tqdm import tqdm
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file


# --- Configuration ---

hf_token = os.getenv("HF_TOKEN")

hf_user = "Rajarshi-Roy-research"
# dataset_name = "Flickr30k_Grounding_Som"
# commit_message = "Updating dataset with image paths"
# img_cols = ["image","wbox_image"]
# image_to_process = "image"

dataset_name = "refcocog-eval-depth"
commit_message = "Updating dataset with images and depth caption"
img_cols = ["image"]
image_to_process = "image_"

# --- Helper Functions ---
def save_image_from_bytes(image_data, idx, image_folder):
    """Saves an image from byte data and returns the saved file path."""
    try:
        image_bytes = image_data.get("bytes", None)
        if image_bytes:
            image = Image_PIL.open(io.BytesIO(image_bytes))
            image_path = os.path.join(image_folder, f"image_{idx}.jpg")
            # image_path = os.path.join(f"{image_folder}__", f"image_{idx}.jpg")
            image.save(image_path, format="JPEG")
            print(f"✅ Saved: {image_path}")
            return image_path
        else:
            print(f"⚠️ Skipped row {idx}: No bytes found--")
            return None
    except Exception as e:
        print(f"❌ Error saving image at row {idx}: {e}")
        return None


# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
# ds = load_dataset(f"{hf_user}/{dataset_name}")
ds = load_dataset(f"{hf_user}/{dataset_name}",revision="d9f8954a6a74fc037f678b1037b7f6e2ec499ed1")
df = ds["train"].to_pandas()


# print("checking")
for img_col in img_cols:
    # Save images and update DataFrame
    for i, row in df.iterrows():
        # print("checking",i,row[img_col])
        if isinstance(row[img_col], dict):
            df.at[i, img_col] = save_image_from_bytes(row[img_col], i, img_col)





✅ Saved: image/image_0.jpg
✅ Saved: image/image_1.jpg
✅ Saved: image/image_2.jpg
✅ Saved: image/image_3.jpg
✅ Saved: image/image_4.jpg
✅ Saved: image/image_5.jpg
✅ Saved: image/image_6.jpg
✅ Saved: image/image_7.jpg
✅ Saved: image/image_8.jpg
✅ Saved: image/image_9.jpg
✅ Saved: image/image_10.jpg
✅ Saved: image/image_11.jpg
✅ Saved: image/image_12.jpg
✅ Saved: image/image_13.jpg
✅ Saved: image/image_14.jpg
✅ Saved: image/image_15.jpg
✅ Saved: image/image_16.jpg
✅ Saved: image/image_17.jpg
✅ Saved: image/image_18.jpg
✅ Saved: image/image_19.jpg
✅ Saved: image/image_20.jpg
✅ Saved: image/image_21.jpg
✅ Saved: image/image_22.jpg
✅ Saved: image/image_23.jpg
✅ Saved: image/image_24.jpg
✅ Saved: image/image_25.jpg
✅ Saved: image/image_26.jpg
✅ Saved: image/image_27.jpg
✅ Saved: image/image_28.jpg
✅ Saved: image/image_29.jpg
✅ Saved: image/image_30.jpg
✅ Saved: image/image_31.jpg
✅ Saved: image/image_32.jpg
✅ Saved: image/image_33.jpg
✅ Saved: image/image_34.jpg
✅ Saved: image/image_35.jpg
✅ 

In [12]:
df

,id,caption,bbox,bbox_area,bbox_id,ori_category_id,image_id,height,width,file_name,is_rewrite,split,image,depth_caption
0,4381,A pair of tortillas containing a visible filli...,[300.25585936 239.42510986 183.92150884 133.25...,24508.697297,o365_24196196,o365_172,o365_489077,512,769,sampled_images/objects365_v1_00489077.jpg,True,val,image/image_0.jpg,Closest: a couple\n----\nMid Range: a bar\n---...
1,3385,"The closed-top, white bucket-shaped item, posi...",[647.26123049 556.64379886 98.25061038 106.61...,10474.736003,o365_3289057,o365_49,o365_1040341,768,1024,sampled_images/objects365_v2_01040341.jpg,True,val,image/image_1.jpg,Closest: a man\n----\nMid Range: tents\n----\n...
2,27674,The pink and white surfboard is curretnly ride...,[120.05 163.64 232.97 77.18],17980.624600,coco_650459,refcoco_42,coco_449136,318,640,sampled_images/COCO_train2014_000000449136.jpg,True,val,image/image_2.jpg,Closest: a man\n----\nMid Range: a surfer\n---...
3,44294,"The man wearing a cheerful Santa hat, characte...",[240.81 14.21 275.38 370.15],101931.907000,coco_2165449,refcoco_1,coco_242583,427,640,sampled_images/COCO_train2014_000000242583.jpg,True,val,image/image_3.jpg,Closest: a man\n----\nMid Range: a hand\n----\...
4,29717,The pizza adorned with chunks of white topping...,[143.32 283.76 353.97 139.47],49368.195900,coco_1074038,refcoco_59,coco_70415,429,640,sampled_images/COCO_train2014_000000070415.jpg,False,val,image/image_4.jpg,Closest: a pizza\n----\nMid Range: a person\n-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12011,The green purse hanging on the wall close to t...,[232.43835453 416.28540035 57.75231928 61.99...,3580.129160,o365_27275769,o365_14,o365_2053317,982,1024,sampled_images/objects365_v2_02053317.jpg,True,val,image/image_395.jpg,Closest: two motorcycles\n----\nMid Range: a m...
396,36631,"The man standing in the background, behind the...",[363.87 211.18 116.49 306.33],35684.381700,coco_500529,refcoco_1,coco_242213,640,512,sampled_images/COCO_train2014_000000242213.jpg,False,val,image/image_396.jpg,Closest: a baseball pitcher\n----\nMid Range: ...
397,18231,The skier clad in a red jacket is standing wit...,[474.02 105.55 130.5 316.65],41322.825000,coco_512212,refcoco_1,coco_271641,427,640,sampled_images/COCO_train2014_000000271641.jpg,False,val,image/image_397.jpg,Closest: people\n----\nMid Range: a silhouette...
398,5846,A transparent drinking vessel containing a yel...,[284.04260255 188.13116452 100.05944822 199.01...,19913.766167,o365_9095135,o365_11,o365_476705,683,512,sampled_images/objects365_v1_00476705.jpg,False,val,image/image_398.jpg,Closest: candles\n----\nMid Range: a plate of ...


In [13]:
# pred_bbox_base_qwen_vl = df["pred_bbox_base_qwen_vl"]
df["pred_bbox_base_qwen_vl"]  =pred_bbox_base_qwen_vl

In [14]:
df

,id,caption,bbox,bbox_area,bbox_id,ori_category_id,image_id,height,width,file_name,is_rewrite,split,image,depth_caption,pred_bbox_base_qwen_vl
0,4381,A pair of tortillas containing a visible filli...,[300.25585936 239.42510986 183.92150884 133.25...,24508.697297,o365_24196196,o365_172,o365_489077,512,769,sampled_images/objects365_v1_00489077.jpg,True,val,image/image_0.jpg,Closest: a couple\n----\nMid Range: a bar\n---...,"[{'bbox_2d': [0, 195, 638, 504], 'label': 'A p..."
1,3385,"The closed-top, white bucket-shaped item, posi...",[647.26123049 556.64379886 98.25061038 106.61...,10474.736003,o365_3289057,o365_49,o365_1040341,768,1024,sampled_images/objects365_v2_01040341.jpg,True,val,image/image_1.jpg,Closest: a man\n----\nMid Range: tents\n----\n...,"[{'bbox_2d': [178, 615, 304, 756], 'label': 'c..."
2,27674,The pink and white surfboard is curretnly ride...,[120.05 163.64 232.97 77.18],17980.624600,coco_650459,refcoco_42,coco_449136,318,640,sampled_images/COCO_train2014_000000449136.jpg,True,val,image/image_2.jpg,Closest: a man\n----\nMid Range: a surfer\n---...,"[{'bbox_2d': [123, 159, 280, 234], 'label': 'T..."
3,44294,"The man wearing a cheerful Santa hat, characte...",[240.81 14.21 275.38 370.15],101931.907000,coco_2165449,refcoco_1,coco_242583,427,640,sampled_images/COCO_train2014_000000242583.jpg,True,val,image/image_3.jpg,Closest: a man\n----\nMid Range: a hand\n----\...,"[{'bbox_2d': [0, 91, 438, 415], 'label': 'The ..."
4,29717,The pizza adorned with chunks of white topping...,[143.32 283.76 353.97 139.47],49368.195900,coco_1074038,refcoco_59,coco_70415,429,640,sampled_images/COCO_train2014_000000070415.jpg,False,val,image/image_4.jpg,Closest: a pizza\n----\nMid Range: a person\n-...,"[{'bbox_2d': [145, 273, 506, 415], 'label': 'p..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12011,The green purse hanging on the wall close to t...,[232.43835453 416.28540035 57.75231928 61.99...,3580.129160,o365_27275769,o365_14,o365_2053317,982,1024,sampled_images/objects365_v2_02053317.jpg,True,val,image/image_395.jpg,Closest: two motorcycles\n----\nMid Range: a m...,"[{'bbox_2d': [397, 491, 596, 758], 'label': 'T..."
396,36631,"The man standing in the background, behind the...",[363.87 211.18 116.49 306.33],35684.381700,coco_500529,refcoco_1,coco_242213,640,512,sampled_images/COCO_train2014_000000242213.jpg,False,val,image/image_396.jpg,Closest: a baseball pitcher\n----\nMid Range: ...,"[{'bbox_2d': [15, 67, 478, 603], 'label': 'The..."
397,18231,The skier clad in a red jacket is standing wit...,[474.02 105.55 130.5 316.65],41322.825000,coco_512212,refcoco_1,coco_271641,427,640,sampled_images/COCO_train2014_000000271641.jpg,False,val,image/image_397.jpg,Closest: people\n----\nMid Range: a silhouette...,"[{'bbox_2d': [475, 98, 610, 415], 'label': 'Th..."
398,5846,A transparent drinking vessel containing a yel...,[284.04260255 188.13116452 100.05944822 199.01...,19913.766167,o365_9095135,o365_11,o365_476705,683,512,sampled_images/objects365_v1_00476705.jpg,False,val,image/image_398.jpg,Closest: candles\n----\nMid Range: a plate of ...,"[{'bbox_2d': [0, 365, 481, 672], 'label': 'A t..."


In [5]:


import ast

def get_parsed_bounding_boxes(response):
    """
    Extracts and parses markdown-formatted JSON containing bounding boxes.

    Args:
        response (str): A string containing the JSON block, possibly within markdown fences.

    Returns:
        list: A list of dictionaries with keys 'bbox_2d' and 'label'.
    """
    # Step 1: Remove markdown fences
    lines = response.splitlines()
    for i, line in enumerate(lines):
        if line.strip() == "```json":
            response = "\n".join(lines[i + 1:])
            response = response.split("```")[0]
            break

    # Step 2: Try parsing using ast.literal_eval
    try:
        parsed = ast.literal_eval(response)
    except Exception as e:
        print("Initial parsing failed, attempting recovery...")
        end_idx = response.rfind('"}') + len('"}')
        truncated_text = response[:end_idx] + "]"
        try:
            parsed = ast.literal_eval(truncated_text)
        except Exception as e2:
            raise ValueError("Could not parse JSON even after recovery.") from e2

    return parsed


def inference(img_url, prompt, system_prompt="You are a helpful assistant", max_new_tokens=1024):
  image = Image_PIL.open(img_url)
  messages = [
    {
      "role": "system",
      "content": system_prompt
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        },
        {
          "image": img_url
        }
      ]
    }
  ]
  
  text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  print("input:\n",text)
  inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt").to('cuda')


  # Patch if missing
  if "image_grid_thw" not in inputs:
      inputs["image_grid_thw"] = torch.tensor([[14, 14, 1]], device='cuda')
      
  with torch.no_grad():

    output_ids = model.generate(**inputs, max_new_tokens=1024)
  
  generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
  output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
  print("output:\n",output_text[0])

  input_height = inputs['image_grid_thw'][0][1]*14
  input_width = inputs['image_grid_thw'][0][2]*14

  return output_text[0], input_height, input_width




In [6]:
df

,id,caption,bbox,bbox_area,bbox_id,ori_category_id,image_id,height,width,file_name,is_rewrite,split,image,depth_caption,pred_bbox_base_qwen_vl
0,4381,A pair of tortillas containing a visible filli...,[300.25585936 239.42510986 183.92150884 133.25...,24508.697297,o365_24196196,o365_172,o365_489077,512,769,sampled_images/objects365_v1_00489077.jpg,True,val,image/image_0.jpg,Closest: a couple\n----\nMid Range: a bar\n---...,"[{'bbox_2d': [0, 195, 638, 504], 'label': 'A p..."
1,3385,"The closed-top, white bucket-shaped item, posi...",[647.26123049 556.64379886 98.25061038 106.61...,10474.736003,o365_3289057,o365_49,o365_1040341,768,1024,sampled_images/objects365_v2_01040341.jpg,True,val,image/image_1.jpg,Closest: a man\n----\nMid Range: tents\n----\n...,"[{'bbox_2d': [178, 615, 304, 756], 'label': 'c..."
2,27674,The pink and white surfboard is curretnly ride...,[120.05 163.64 232.97 77.18],17980.624600,coco_650459,refcoco_42,coco_449136,318,640,sampled_images/COCO_train2014_000000449136.jpg,True,val,image/image_2.jpg,Closest: a man\n----\nMid Range: a surfer\n---...,"[{'bbox_2d': [123, 159, 280, 234], 'label': 'T..."
3,44294,"The man wearing a cheerful Santa hat, characte...",[240.81 14.21 275.38 370.15],101931.907000,coco_2165449,refcoco_1,coco_242583,427,640,sampled_images/COCO_train2014_000000242583.jpg,True,val,image/image_3.jpg,Closest: a man\n----\nMid Range: a hand\n----\...,"[{'bbox_2d': [0, 91, 438, 415], 'label': 'The ..."
4,29717,The pizza adorned with chunks of white topping...,[143.32 283.76 353.97 139.47],49368.195900,coco_1074038,refcoco_59,coco_70415,429,640,sampled_images/COCO_train2014_000000070415.jpg,False,val,image/image_4.jpg,Closest: a pizza\n----\nMid Range: a person\n-...,"[{'bbox_2d': [145, 273, 506, 415], 'label': 'p..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12011,The green purse hanging on the wall close to t...,[232.43835453 416.28540035 57.75231928 61.99...,3580.129160,o365_27275769,o365_14,o365_2053317,982,1024,sampled_images/objects365_v2_02053317.jpg,True,val,image/image_395.jpg,Closest: two motorcycles\n----\nMid Range: a m...,"[{'bbox_2d': [397, 491, 596, 758], 'label': 'T..."
396,36631,"The man standing in the background, behind the...",[363.87 211.18 116.49 306.33],35684.381700,coco_500529,refcoco_1,coco_242213,640,512,sampled_images/COCO_train2014_000000242213.jpg,False,val,image/image_396.jpg,Closest: a baseball pitcher\n----\nMid Range: ...,"[{'bbox_2d': [15, 67, 478, 603], 'label': 'The..."
397,18231,The skier clad in a red jacket is standing wit...,[474.02 105.55 130.5 316.65],41322.825000,coco_512212,refcoco_1,coco_271641,427,640,sampled_images/COCO_train2014_000000271641.jpg,False,val,image/image_397.jpg,Closest: people\n----\nMid Range: a silhouette...,"[{'bbox_2d': [475, 98, 610, 415], 'label': 'Th..."
398,5846,A transparent drinking vessel containing a yel...,[284.04260255 188.13116452 100.05944822 199.01...,19913.766167,o365_9095135,o365_11,o365_476705,683,512,sampled_images/objects365_v1_00476705.jpg,False,val,image/image_398.jpg,Closest: candles\n----\nMid Range: a plate of ...,"[{'bbox_2d': [0, 365, 481, 672], 'label': 'A t..."


In [37]:



from PIL import Image as Image_PIL

def process_and_plot_bboxes(df, idx, inference_fn=inference,with_depth=False):
    """
    Processes an image from a DataFrame at a given index, runs inference, and plots bounding boxes.

    Args:
        df (pd.DataFrame): DataFrame containing 'image' and 'caption' columns.
        idx (int): Index of the row to process.
        inference_fn (function): Function that takes (image_path, prompt) and returns (response, input_height, input_width).
        plot_fn (function): Function that plots bounding boxes on the image.
    
    Returns:
        parsed_json (list): List of parsed bounding box dicts.
    """
    image_path = df["image"][idx]
    caption = df["caption"][idx]
    depth_caption = df["depth_caption"][idx]
    
    if not with_depth:

        prompt = f"""
You are given an image along with a depth-based caption to help you understand the spatial layout.

Now, based on the following query:
"{caption}"

Your task is to outline **only the region(s) explicitly mentioned in the query**.

Return the result as a list of bounding box coordinates in **JSON format**, where each item contains:
- "bbox_2d": [x_min, y_min, x_max, y_max] (integers)
- "label": a short description of the outlined object

Example output:
[
  {{
    "bbox_2d": [100, 150, 200, 250],
    "label": "the red car"
  }}
]

Only include regions that are directly referenced in the query.
"""

    else:
       prompt = f"""
You are given an image along with a depth-based caption to help you understand the spatial layout.

Depth-related caption (for spatial understanding):
"{depth_caption}"

Now, based on the following query:
Main Query:"{caption}"

Your task is to outline **only the region explicitly mentioned in the Main Query**.

Return the result as a list of bounding box coordinates in **JSON format**, where each item contains: (There will be only 1 item)
- "bbox_2d": [x_min, y_min, x_max, y_max] (integers)
- "label": a short description of the outlined object

Example output:
[
  {{
    "bbox_2d": [100, 150, 200, 250],
    "label": "the red car"
  }}
]

Only include regions that are directly referenced in the query.
"""


    response, input_height, input_width = inference_fn(image_path, prompt)

    # Load and resize image
    image = Image_PIL.open(image_path)
    print(f"Original size: {image.size}")
    image.thumbnail([640, 640], Image_PIL.Resampling.LANCZOS)

    # Plot and return parsed bounding boxes
    parsed_json = get_parsed_bounding_boxes(response)
    
    
    return parsed_json



In [38]:


process_and_plot_bboxes(df,2, inference_fn=inference,with_depth=True)


input:
 <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user

You are given an image along with a depth-based caption to help you understand the spatial layout.

Depth-related caption (for spatial understanding):
"Closest: a man
----
Mid Range: a surfer
----
Farthest: a surfer
----
"

Now, based on the following query:
Main Query:"The pink and white surfboard is curretnly riden by a man wearing a blue shirt."

Your task is to outline **only the region explicitly mentioned in the Main Query**.

Return the result as a list of bounding box coordinates in **JSON format**, where each item contains: (There will be only 1 item)
- "bbox_2d": [x_min, y_min, x_max, y_max] (integers)
- "label": a short description of the outlined object

Example output:
[
  {
    "bbox_2d": [100, 150, 200, 250],
    "label": "the red car"
  }
]

Only include regions that are directly referenced in the query.
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant



output:
 [
  {
    "bbox_2d": [249, 117, 430, 238], 
    "label": "man riding surfboard"
  }
]
Original size: (640, 318)


[{'bbox_2d': [249, 117, 430, 238], 'label': 'man riding surfboard'}]

In [40]:
# df

In [41]:
from datasets import Dataset

# Convert your pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)


In [42]:

len(bbox_results)

400

In [43]:
# # Add bounding box predictions
bbox_results = []

for idx in tqdm(range(len(dataset)), desc="Processing bounding boxes"):
    try:
        bboxes = process_and_plot_bboxes(df, idx, inference_fn=inference,with_depth=True)
    except Exception as e:
        print(f"Failed on index {idx}: {e}")
        bboxes = None
    bbox_results.append(bboxes)
    
# Add as a new column
dataset = dataset.add_column("pred_bbox_with_DEPTH_qwen_vl", bbox_results)


Processing bounding boxes:   0%|          | 0/400 [00:00<?, ?it/s]

input:
 <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user

You are given an image along with a depth-based caption to help you understand the spatial layout.

Depth-related caption (for spatial understanding):
"Closest: a couple
----
Mid Range: a bar
----
Farthest: a bartender
----
"

Now, based on the following query:
Main Query:"A pair of tortillas containing a visible filling are positioned next to a heap of rice on a round plate."

Your task is to outline **only the region explicitly mentioned in the Main Query**.

Return the result as a list of bounding box coordinates in **JSON format**, where each item contains: (There will be only 1 item)
- "bbox_2d": [x_min, y_min, x_max, y_max] (integers)
- "label": a short description of the outlined object

Example output:
[
  {
    "bbox_2d": [100, 150, 200, 250],
    "label": "the red car"
  }
]

Only include regions that are directly referenced in the query.
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|

Processing bounding boxes:   0%|          | 1/400 [00:03<26:32,  3.99s/it]

output:
 ```json
[
  {
    "bbox_2d": [296, 238, 474, 370],
    "label": "tortillas with visible filling"
  },
  {
    "bbox_2d": [468, 268, 558, 353],
    "label": "heap of rice"
  }
]
```
Original size: (769, 512)
input:
 <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user

You are given an image along with a depth-based caption to help you understand the spatial layout.

Depth-related caption (for spatial understanding):
"Closest: a man
----
Mid Range: tents
----
Farthest: a food market at night
----
"

Now, based on the following query:
Main Query:"The closed-top, white bucket-shaped item, positioned among numerous cooking tools and food receptacles at the street food stand."

Your task is to outline **only the region explicitly mentioned in the Main Query**.

Return the result as a list of bounding box coordinates in **JSON format**, where each item contains: (There will be only 1 item)
- "bbox_2d": [x_min, y_min, x_max, y_max] (integers)
- "label": a short d

Processing bounding boxes:   0%|          | 2/400 [00:06<20:53,  3.15s/it]

...

output:
 ```json
[
  {
    "bbox_2d": [476, 98, 588, 419], 
    "label": "person wearing red jacket and white hat"
  },
  {
    "bbox_2d": [371, 78, 492, 354], 
    "label": "person wearing blue jacket and black hat"
  },
  {
    "bbox_2d": [46, 111, 162, 419], 
    "label": "person wearing black jacket and black hat"
  }
]
```
Original size: (640, 427)
input:
 <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user

You are given an image along with a depth-based caption to help you understand the spatial layout.

Depth-related caption (for spatial understanding):
"Closest: candles
----
Mid Range: a plate of food
----
Farthest: a table
----
"

Now, based on the following query:
Main Query:"A transparent drinking vessel containing a yellow-orange liquid located centrally on the table in the foreground."

Your task is to outline **only the region explicitly mentioned in the Main Query**.

Return the result as a list of bounding box coordinates in **JSON format**, where

Processing bounding boxes: 100%|█████████▉| 399/400 [31:55<00:03,  3.20s/it]

output:
 ```json
[
  {
    "bbox_2d": [137, 400, 280, 476], 
    "label": "a transparent drinking vessel containing a yellow-orange liquid"
  }
]
```
Original size: (512, 683)
input:
 <|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user

You are given an image along with a depth-based caption to help you understand the spatial layout.

Depth-related caption (for spatial understanding):
"Closest: a car
----
Mid Range: a car
----
Farthest: a dog
----
"

Now, based on the following query:
Main Query:"A tamed ruminant creature is nestled in the storage area of a car, usually seen in a meadow instead of vehicular environments."

Your task is to outline **only the region explicitly mentioned in the Main Query**.

Return the result as a list of bounding box coordinates in **JSON format**, where each item contains: (There will be only 1 item)
- "bbox_2d": [x_min, y_min, x_max, y_max] (integers)
- "label": a short description of the outlined object

Example output:
[
  {
  

Processing bounding boxes: 100%|██████████| 400/400 [31:57<00:00,  4.79s/it]

output:
 ```json
[
  {
    "bbox_2d": [360, 379, 849, 507], "label": "tamed ruminant creature in the trunk"
  }
]
```
Original size: (1024, 768)


In [44]:
dataset[1]

{'id': 3385,
 'caption': 'The closed-top, white bucket-shaped item, positioned among numerous cooking tools and food receptacles at the street food stand.',
 'bbox': '[647.26123049 556.64379886  98.25061038 106.61242675]',
 'bbox_area': 10474.736002562324,
 'bbox_id': 'o365_3289057',
 'ori_category_id': 'o365_49',
 'image_id': 'o365_1040341',
 'height': 768,
 'width': 1024,
 'file_name': 'sampled_images/objects365_v2_01040341.jpg',
 'is_rewrite': True,
 'split': 'val',
 'image': 'image/image_1.jpg',
 'depth_caption': 'Closest: a man\n----\nMid Range: tents\n----\nFarthest: a food market at night\n----\n',
 'pred_bbox_base_qwen_vl': [{'bbox_2d': [178, 615, 304, 756],
   'label': 'closed-top, white bucket-shaped item'}],
 'pred_bbox_with_DEPTH_qwen_vl': [{'bbox_2d': [654, 547, 753, 653],
   'label': 'closed-top, white bucket-shaped item'}]}

In [45]:


# Push the cleaned dataset to Hugging Face Hub
hf_user = "Rajarshi-Roy-research"  # Your Hugging Face username
dataset_name = "refcocog-eval-depth"  # The dataset name
commit_message = "Adding refcocog data with base eval "


# Cast the 'image' column to proper Image feature
dataset = dataset.cast_column("image", Image())

try:
    dataset.push_to_hub(f"{hf_user}/{dataset_name}", commit_message=commit_message, token=hf_token, private=False)
    print(f"Dataset {hf_user}/{dataset_name} uploaded.")
except Exception as e:
    print(f"Error pushing to Hugging Face Hub: {e}")



Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Dataset Rajarshi-Roy-research/refcocog-eval-depth uploaded.


In [16]:
dataset["image"]

NameError: name 'dataset' is not defined

In [61]:

import numpy as np

def compute_iou(pred_mask: np.ndarray, gt_mask: np.ndarray) -> float:
    """
    Compute Intersection over Union (IoU) between predicted and ground truth masks.

    Args:
        pred_mask (np.ndarray): Binary mask predicted by the model (shape: H x W).
        gt_mask (np.ndarray): Ground truth binary mask (shape: H x W).

    Returns:
        float: IoU score between 0 and 1.
    """
    assert pred_mask.shape == gt_mask.shape, "Mask shapes must match"

    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()

    if union == 0:
        return 1.0 if intersection == 0 else 0.0  # Edge case: both masks empty

    iou = intersection / union
    return iou


In [62]:
import numpy as np

In [63]:
idx = 1

gt_mask = np.fromstring(dataset[idx]["bbox"].strip('[]'), sep=' ').tolist()
pred_bbox_base_qwen_vl = dataset[idx]["pred_bbox_base_qwen_vl"][0]["bbox_2d"]

compute_iou

<function __main__.compute_iou(pred_mask: numpy.ndarray, gt_mask: numpy.ndarray) -> float>

In [65]:
def compute_iou_bbox(boxA, boxB):
    """
    Compute IoU between two bounding boxes: [x, y, w, h]

    Args:
        boxA (list or array): Ground truth bounding box
        boxB (list or array): Predicted bounding box

    Returns:
        float: IoU score
    """
    # Convert to [x1, y1, x2, y2]
    xA1, yA1, wA, hA = boxA
    xA2, yA2 = xA1 + wA, yA1 + hA

    xB1, yB1, wB, hB = boxB
    xB2, yB2 = xB1 + wB, yB1 + hB

    # Compute intersection box
    xI1, yI1 = max(xA1, xB1), max(yA1, yB1)
    xI2, yI2 = min(xA2, xB2), min(yA2, yB2)

    inter_w = max(0, xI2 - xI1)
    inter_h = max(0, yI2 - yI1)
    intersection = inter_w * inter_h

    areaA = wA * hA
    areaB = wB * hB
    union = areaA + areaB - intersection

    if union == 0:
        return 0.0

    return intersection / union


In [68]:
ious = []

for idx in range(len(dataset)):
    # Get ground truth box
    gt_entry = dataset[idx].get("bbox")
    pred_entry = dataset[idx].get("pred_bbox_base_qwen_vl")

    # Skip if missing data
    if not gt_entry or not pred_entry or len(pred_entry) == 0:
        continue

    # Parse ground truth bbox
    gt_box = np.fromstring(gt_entry.strip("[]"), sep=" ").tolist()

    # Parse predicted bbox
    pred_box = pred_entry[0].get("bbox_2d")
    if not pred_box:
        continue

    # Compute IoU
    iou = compute_iou_bbox(gt_box, pred_box)
    ious.append(iou)

# Final result
if ious:
    mean_iou = sum(ious) / len(ious)
    print(f"Mean IoU over dataset: {mean_iou:.4f} ({len(ious)} valid samples)")
else:
    print("No valid predictions to compute IoU.")


Mean IoU over dataset: 0.1979 (399 valid samples)


In [69]:
ious = []

for idx in range(len(dataset)):
    # Get ground truth box
    gt_entry = dataset[idx].get("bbox")
    pred_entry = dataset[idx].get("pred_bbox_with_DEPTH_qwen_vl")

    # Skip if missing data
    if not gt_entry or not pred_entry or len(pred_entry) == 0:
        continue

    # Parse ground truth bbox
    gt_box = np.fromstring(gt_entry.strip("[]"), sep=" ").tolist()

    # Parse predicted bbox
    pred_box = pred_entry[0].get("bbox_2d")
    if not pred_box:
        continue

    # Compute IoU
    iou = compute_iou_bbox(gt_box, pred_box)
    ious.append(iou)

# Final result
if ious:
    mean_iou = sum(ious) / len(ious)
    print(f"Mean IoU over dataset: {mean_iou:.4f} ({len(ious)} valid samples)")
else:
    print("No valid predictions to compute IoU.")


Mean IoU over dataset: 0.1582 (384 valid samples)


In [67]:
len(ious)

243